In [ ]:
import pandas as pd
import numpy as np
import math

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import preprocessing
from sklearn.linear_model import SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
file_path = '/content/drive/MyDrive/kb차차차/최종 데이터.csv'

In [ ]:
df = pd.read_csv(file_path, engine = 'python', encoding = 'cp949')

In [ ]:
df = df.drop(columns = ['Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12', 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18'])

In [ ]:
df['제조사'] = df['제조사'].replace('쌍용 ', '쌍용')

In [ ]:
df['모델'] = df['모델'].replace('코란도 ', '코란도')
df['모델'] = df['모델'].replace('렉스턴 ', '렉스턴')
df['모델'] = df['모델'].replace('3.8', 'G90')
df['모델'] = df['모델'].replace('쉐보레 말리부', '말리부')
df['모델'] = df['모델'].replace(' 쉐보레 말리부', '말리부')
df['모델'] = df['모델'].replace('쉐보레 카마로', '카마로')
df['모델'] = df['모델'].replace('토스카 ', '토스카')
df['모델'] = df['모델'].replace('쉐보레 콜벳', '콜벳')
df['모델'] = df['모델'].replace('쉐보레 트래버스', '트래버스')
df['모델'] = df['모델'].replace('쉐보레 임팔라', '임팔라')
df['모델'] = df['모델'].replace('쉐보레 캡티바', '캡티바')
df['모델'] = df['모델'].replace('쉐보레 올란도', '올란도')
df['모델'] = df['모델'].replace('쉐보레 크루즈', '크루즈')
df['모델'] = df['모델'].replace('쉐보레 아베오', '아베오')
df['모델'] = df['모델'].replace('쉐보레 트랙스', '트랙스')
df['모델'] = df['모델'].replace('쉐보레 이쿼녹스', '이쿼녹스')
df['모델'] = df['모델'].replace('쉐보레 트레일블레이저', '트레일블레이저')
df['모델'] = df['모델'].replace('쉐보레 스파크', '스파크')
df['모델'] = df['모델'].replace('쉐보레 볼트', '볼트')
df['모델'] = df['모델'].replace('제네시스 쿠페', '쿠페')
df['모델'] = df['모델'].replace('펠리세이드', '팰리세이드')
df['모델'] = df['모델'].replace('투', '투스카니')
df['모델'] = df['모델'].replace('베라쉐보레 크루즈', '베라크루즈')

In [ ]:
df

,제조사,모델,차종,가격(만원),연식,주행거리(km),연료,변속기,연비(km),배기량(cc)
0,기아,모닝,경차,210,2008,100000,가솔린,오토,0.0,991.0
1,쌍용,티볼리,SUV,1860,2019,4427,가솔린,오토,12.0,4427.0
2,현대,쿠페,스포츠카,870,2010,142395,가솔린,오토,0.0,3778.9
3,제네시스,BH330,대형,990,2009,171709,가솔린,오토,0.0,3300.0
4,현대,제네시스,대형,990,2009,171709,가솔린,오토,0.0,3300.0
...,...,...,...,...,...,...,...,...,...,...
65871,기아,스포티지,SUV,610,2009,134118,가솔린,오토,0.0,0.0
65872,기아,스포티지,SUV,250,2006,290000,가솔린,오토,0.0,0.0
65873,기아,스포티지,SUV,300,2009,238700,디젤,오토,15.0,0.0
65874,기아,스포티지,SUV,350,2006,119506,디젤,오토,0.0,0.0


In [ ]:
# 원본

car_arr = ['제네시스 쿠페', '제네시스', '그랜저', '에쿠스', '펠리세이드', '베라크루즈', '팰리세이드', '베라쉐보레 크루즈', '아슬란', '맥스크루즈',
           '쏘나타', '아반떼', '다이너스티', '갤로퍼', '테라칸', '트라제XG', '투스카니', '싼타페', '투', '스타렉스', '스타리아', 'i30', '벨로스터',
           '투싼', '코나', 'i40', '엘란트라', '마르샤', '아이오닉', '그라나다', '티뷰론', '엑센트', '베르나', '클릭', '베뉴', '뉴엘란트라', '포니',
           '아이오닉5', '넥쏘', '아토스', '엑셀', '모닝', 'K7', '프라이드', 'K9', '모하비', '카니발', '오피러스', '아만티', 'K8', 'K5', '로체', '스팅어',
           '쏘렌토', '엔터프라이즈', '쏘울', '포르테', 'K3', '포텐샤', '카렌스', '크레도스', '스포티지', '옵티마', '레토나', '콩코드', '엑스트렉', '엘란',
           '쎄라토', '스토닉', '셀토스', '니로', '세피아', '스펙트라', '레이', '모닝밴', '레이밴', '비스토', '카스타', '캐피탈', '티볼리', '무쏘', '렉스턴',
           '코란도', '카이런', '코란도', '액티언', '체어맨', '렉스턴', '로디우스', 'BH330', 'BH380', 'GP500', 'G330', 'G380', 'G90', 'EQ900', 'G80', 'BH460',
           '3.8', 'GV70', 'GV80', 'G70', 'GP380', '토스카', '쉐보레 말리부', '쉐보레 카마로', '카마로', '쉐보레 콜벳', '쉐보레 트래버스', '쉐보레 임팔라', 
           '스테이츠맨', '베리타스', '아카디아', '알페온', '윈스톰', '쉐보레 캡티바', '쉐보레 올란도', '라세티', '쉐보레 크루즈', '레조', 'G2X', '쉐보레 말리부',
           '토스카', '프린스', '쉐보레 아베오', '쉐보레 트랙스', '쉐보레 이쿼녹스', '젠트라', '쉐보레 볼트', '쉐보레 트레일블레이저', '쉐보레 스파크', '마티즈', 
           '스파크', '티코', '라보', '다마스', 'QM6', 'SM7', 'SM5', 'QM5', 'SM3', 'SM6', 'QM3', '클리오', 'XM3', '캡처', '조에', '트위지']

In [ ]:
# 140개_수정
car_arr = ['쿠페', '제네시스', '그랜저', '에쿠스', '베라크루즈', '팰리세이드', '아슬란', '맥스크루즈', '쏘나타', '아반떼', 
           '다이너스티', '갤로퍼', '테라칸', '트라제XG', '싼타페', '투스카니', '스타렉스', '스타리아', 'i30', 
           '벨로스터', '투싼', '코나', 'i40', '엘란트라', '마르샤', '아이오닉', '그라나다', '티뷰론', '엑센트', 
           '베르나', '클릭', '베뉴', '뉴엘란트라', '포니', '아이오닉5', '넥쏘', '아토스', '엑셀', '모닝', 
           'K7', '프라이드', 'K9', '모하비', '카니발', '오피러스', '아만티', 'K8', 'K5', '로체', 
           '스팅어', '쏘렌토', '엔터프라이즈', '쏘울', '포르테', 'K3', '포텐샤', '카렌스', '크레도스', '스포티지', 
           '옵티마', '레토나', '콩코드', '엑스트렉', '엘란', '쎄라토', '스토닉', '셀토스', '니로', '세피아', 
           '스펙트라', '레이', '모닝밴', '레이밴', '비스토', '카스타', '캐피탈', '티볼리', '무쏘', '렉스턴',
           '코란도', '카이런', '코란도', '액티언', '체어맨', '렉스턴', '로디우스', 'BH330', 'BH380', 'GP500', 
           'G330', 'G380', 'G90', 'EQ900', 'G80', 'BH460', 'GV70', 'GV80', 'G70', 'GP380', 
           '말리부', '카마로', '콜벳', '트래버스', '임팔라', '스테이츠맨', '베리타스', '아카디아', '알페온', '윈스톰', 
           '캡티바', '올란도', '라세티', '크루즈', '레조', 'G2X', '토스카', '프린스', '아베오', '트랙스', 
           '이쿼녹스', '젠트라', '볼트', '트레일블레이저', '마티즈', '스파크', '티코', '라보', '다마스', 'QM6', 
           'SM7', 'SM5', 'QM5', 'SM3', 'SM6', 'QM3', '클리오', 'XM3', '캡처', '조에', '트위지']

In [ ]:
print(df[df['모델'] == 'SM3'].min().values)
print(df[df['모델'] == 'SM3'].max().values)
print(df[df['모델'] == 'SM3'].mean())

['르노삼성' 'SM3' '준중형' 80 2002 3800 'CNG' 'CVT' 0.0 0.0]
['르노삼성' 'SM3' '준중형' 1350 2020 321552 '전기' '오토' 135.0 2000.0]
가격(만원)         475.498035
연식            2011.702358
주행거리(km)    108237.046169
연비(km)           6.346758
배기량(cc)       1042.377210
dtype: float64


In [ ]:
df[df['가격(만원)'] == df['가격(만원)'].max()]

,제조사,모델,차종,가격(만원),연식,주행거리(km),연료,변속기,연비(km),배기량(cc)
61547,기아,K5,중형,49500,2020,106790,가솔린,오토,0.0,0.0


In [ ]:
df_sm3 = df[df['모델'] == 'SM3']
df_sm3.head()

,제조사,모델,차종,가격(만원),연식,주행거리(km),연료,변속기,연비(km),배기량(cc)
19288,르노삼성,SM3,준중형,380,2014,148129,가솔린,오토,0.0,2000.0
19289,르노삼성,SM3,준중형,230,2009,156718,가솔린,오토,0.0,2000.0
23227,르노삼성,SM3,준중형,430,2014,102366,가솔린,오토,15.0,1998.0
23228,르노삼성,SM3,준중형,450,2010,150000,가솔린,오토,0.0,1998.0
23229,르노삼성,SM3,준중형,290,2009,110000,가솔린,오토,0.0,1998.0


In [ ]:
df_sm3[df_sm3['가격(만원)'] == df_sm3['가격(만원)'].max()]

,제조사,모델,차종,가격(만원),연식,주행거리(km),연료,변속기,연비(km),배기량(cc)
54391,르노삼성,SM3,준중형,1350,2020,35466,전기,오토,0.0,0.0
54392,르노삼성,SM3,준중형,1350,2019,19729,전기,오토,0.0,0.0


In [ ]:
df_sm3[df_sm3['가격(만원)'] == df_sm3['가격(만원)'].min()]

,제조사,모델,차종,가격(만원),연식,주행거리(km),연료,변속기,연비(km),배기량(cc)
33226,르노삼성,SM3,준중형,80,2008,139000,가솔린,오토,0.0,1598.0


In [ ]:
round(df_sm3.mean()).iloc[4]

1042.0

In [ ]:
ms = df[df['모델'] == 'SM3'].min().values
mn = df[df['모델'] == 'SM3'].max().values
ma = round(df[df['모델'] == 'SM3'].mean())

In [ ]:
str(ms), str(mn), str(ma)

("['르노삼성' 'SM3' '준중형' 80 2002 3800 'CNG' 'CVT' 0.0 0.0]",
 "['르노삼성' 'SM3' '준중형' 1350 2020 321552 '전기' '오토' 135.0 2000.0]",
 '가격(만원)         475.0\n연식            2012.0\n주행거리(km)    108237.0\n연비(km)           6.0\n배기량(cc)       1042.0\ndtype: float64')

In [ ]:
def carinfo(models) :
    ms = df[df['모델'] == models].min().values
    mn = df[df['모델'] == models].max().values
    ma = [round(df[df['모델'] == models].mean()).iloc[0], round(df[df['모델'] == models].mean()).iloc[1],
          round(df[df['모델'] == models].mean()).iloc[2], round(df[df['모델'] == models].mean()).iloc[3],
          round(df[df['모델'] == models].mean()).iloc[4]]
    infos = '최고가 정보 :' + str(mn) + '\n' + '최저가 정보 :' + str(ms) + '\n' + '평균가 정보 :' + str(ma)
    return infos

In [ ]:
carinfo('스파크')

"최고가 정보 :['한국GM' '스파크' '소형' 1530 2021 232398 '디젤' '오토' 21.0 1000.0]\n최저가 정보 :['한국GM' '스파크' '경차' 159 2010 10 'LPG' 'CVT' 0.0 0.0]\n평균가 정보 :[598.0, 2015.0, 70663.0, 9.0, 679.0]"

In [ ]:
carinfo('G70')

"최고가 정보 :['제네시스' 'G70' '중형' 5550 2021 220000 '디젤' '오토' 15.2 3342.0]\n최저가 정보 :['제네시스' 'G70' '중형' 1899 2017 78 '가솔린' '오토' 0.0 0.0]\n평균가 정보 :[3675.0, 2019.0, 38305.0, 9.0, 2384.0]"

In [ ]:
car_arr = ['스파크', 'G70', '쏘나타']

In [ ]:
for cars in car_arr :
    infos = carinfo(cars)

infos

"최고가 정보 :['현대' '쏘나타' '중형' 3580 2021 420000 '하이브리드(가솔린)' '오토' 21.0 3000.0]\n최저가 정보 :['현대' '쏘나타' '중형' 129 1991 10 'LPG' '수동' 0.0 0.0]\n평균가 정보 :[1106.0, 2014.0, 109144.0, 4.0, 1382.0]"

In [ ]:
round(df[df['모델'] == '스파크'].mean())

가격(만원)        598.0
연식           2015.0
주행거리(km)    70663.0
연비(km)          9.0
배기량(cc)       679.0
dtype: float64

In [ ]:
sinfo = ['가격 :' + round(df[df['모델'] == '스파크'].mean()).iloc[0] + '만원', '연식 :' + round(df[df['모델'] == '스파크'].mean()).iloc[1],
 '주행거리 :' + round(df[df['모델'] == '스파크'].mean()).iloc[2] + 'km', '연비 :' + round(df[df['모델'] == '스파크'].mean()).iloc[3] + 'km',
 '배기량 :' + round(df[df['모델'] == '스파크'].mean()).iloc[4] + 'cc']

TypeError: ignored

In [ ]:
sinfo = [round(df[df['모델'] == '스파크'].mean()).iloc[0], round(df[df['모델'] == '스파크'].mean()).iloc[1],
        round(df[df['모델'] == '스파크'].mean()).iloc[2], round(df[df['모델'] == '스파크'].mean()).iloc[3],
        round(df[df['모델'] == '스파크'].mean()).iloc[4]]

In [ ]:
str(sinfo)

In [ ]:
# 원본


def query_ans(msg):
    btn_data = msg["data"]

    if btn_data == '현대':
        h_btn1 = BT(text = "제네시스 쿠페", callback_data = "제네시스 쿠페")
        h_btn2 = BT(text = "제네시스", callback_data = "제네시스")
        h_btn3 = BT(text = "그랜저", callback_data = "그랜저")
        h_btn4 = BT(text = "에쿠스", callback_data = "에쿠스")
        h_btn5 = BT(text = "펠리세이드", callback_data = "펠리세이드")
        h_btn6 = BT(text = "베라크루즈", callback_data = "베라크루즈")
        h_btn7 = BT(text = "팰리세이드", callback_data = "팰리세이드")
        h_btn8 = BT(text = "베라쉐보레 크루즈", callback_data = "베라쉐보레 크루즈")
        h_btn9 = BT(text = "아슬란", callback_data = "아슬란")
        h_btn10 = BT(text = "맥스크루즈", callback_data = "맥스크루즈")
        h_btn11 = BT(text = "쏘나타", callback_data = "쏘나타")
        h_btn12 = BT(text = "아반떼", callback_data = "아반떼")
        h_btn13 = BT(text = "다이너스티", callback_data = "다이너스티")
        h_btn14 = BT(text = "갤로퍼", callback_data = "갤로퍼")
        h_btn15 = BT(text = "테라칸", callback_data = "테라칸")
        h_btn16 = BT(text = "트라제XG", callback_data = "트라제XG")
        h_btn17 = BT(text = "투스카니", callback_data = "투스카니")
        h_btn18 = BT(text = "싼타페", callback_data = "싼타페")
        h_btn19 = BT(text = "투", callback_data = "투")
        h_btn20 = BT(text = "스타렉스", callback_data = "스타렉스")
        h_btn21 = BT(text = "스타리아", callback_data = "스타리아")
        h_btn22 = BT(text = "i30", callback_data = "i30")
        h_btn23 = BT(text = "벨로스터", callback_data = "벨로스터")
        h_btn24 = BT(text = "투싼", callback_data = "투싼")
        h_btn25 = BT(text = "코나", callback_data = "코나")
        h_btn26 = BT(text = "i40", callback_data = "i40")
        h_btn27 = BT(text = "엘란트라", callback_data = "엘란트라")
        h_btn28 = BT(text = "마르샤", callback_data = "마르샤")
        h_btn29 = BT(text = "아이오닉", callback_data = "아이오닉")
        h_btn30 = BT(text = "그라나다", callback_data = "그라나다")
        h_btn31 = BT(text = "티뷰론", callback_data = "티뷰론")
        h_btn32 = BT(text = "엑센트", callback_data = "엑센트")
        h_btn33 = BT(text = "베르나", callback_data = "베르나")
        h_btn34 = BT(text = "클릭", callback_data = "클릭")
        h_btn35 = BT(text = "베뉴", callback_data = "베뉴")
        h_btn36 = BT(text = "뉴엘란트라", callback_data = "뉴엘란트라")
        h_btn37 = BT(text = "포니", callback_data = "포니")
        h_btn38 = BT(text = "아이오닉5", callback_data = "아이오닉5")
        h_btn39 = BT(text = "넥쏘", callback_data = "넥쏘")
        h_btn40 = BT(text = "아토스", callback_data = "아토스")
        h_btn41 = BT(text = "엑셀", callback_data = "엑셀")
        mu = MU(inline_keyboard = [[h_btn1, h_btn2], [h_btn3, h_btn4], [h_btn5, h_btn6], [h_btn7, h_btn8],
                                   [h_btn9, h_btn10], [h_btn11, h_btn12], [h_btn13, h_btn14], [h_btn15, h_btn16],
                                   [h_btn17, h_btn18], [h_btn19, h_btn20], [h_btn21, h_btn22], [h_btn23, h_btn24],
                                   [h_btn25, h_btn26], [h_btn27, h_btn28], [h_btn29, h_btn30], [h_btn31, h_btn32],
                                   [h_btn33, h_btn34], [h_btn35, h_btn36], [h_btn37, h_btn38], [h_btn39, h_btn40, h_btn41]])
        bot.sendMessage(id, "현대차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '기아':
        k_btn1 = BT(text = "모닝", callback_data = "모닝")
        k_btn2 = BT(text = "K7", callback_data = "K7")
        k_btn3 = BT(text = "K9", callback_data = "K9")
        k_btn4 = BT(text = "프라이드", callback_data = "프라이드")
        k_btn5 = BT(text = "모하비", callback_data = "모하비")
        k_btn6 = BT(text = "카니발", callback_data = "카니발")
        k_btn7 = BT(text = "오피러스", callback_data = "오피러스")
        k_btn8 = BT(text = "아만티", callback_data = "아만티")
        k_btn9 = BT(text = "K8", callback_data = "K8")
        k_btn10 = BT(text = "K5", callback_data = "K5")
        k_btn11 = BT(text = "로체", callback_data = "로체")
        k_btn12 = BT(text = "스팅어", callback_data = "스팅어")
        k_btn13 = BT(text = "쏘렌토", callback_data = "쏘렌토")
        k_btn14 = BT(text = "엔터프라이즈", callback_data = "엔터프라이즈")
        k_btn15 = BT(text = "쏘울", callback_data = "쏘울")
        k_btn16 = BT(text = "포르테", callback_data = "포르테")
        k_btn17 = BT(text = "K3", callback_data = "K3")
        k_btn18 = BT(text = "포텐샤", callback_data = "포텐샤")
        k_btn19 = BT(text = "카렌스", callback_data = "카렌스")
        k_btn20 = BT(text = "크레도스", callback_data = "크레도스")
        k_btn21 = BT(text = "스포티지", callback_data = "스포티지")
        k_btn22 = BT(text = "옵티마", callback_data = "옵티마")
        k_btn23 = BT(text = "레토나", callback_data = "레토나")
        k_btn24 = BT(text = "콩코드", callback_data = "콩코드")
        k_btn25 = BT(text = "엑스트렉", callback_data = "엑스트렉")
        k_btn26 = BT(text = "엘란", callback_data = "엘란")
        k_btn27 = BT(text = "쎄라토", callback_data = "쎄라토")
        k_btn28 = BT(text = "스토닉", callback_data = "스토닉")
        k_btn29 = BT(text = "셀토스", callback_data = "셀토스")
        k_btn30 = BT(text = "니로", callback_data = "니로")
        k_btn31 = BT(text = "세피아", callback_data = "세피아")
        k_btn32 = BT(text = "스펙트라", callback_data = "스펙트라")
        k_btn33 = BT(text = "레이", callback_data = "레이")
        k_btn34 = BT(text = "모닝밴", callback_data = "모닝밴")
        k_btn35 = BT(text = "레이밴", callback_data = "레이밴")
        k_btn36 = BT(text = "비스토", callback_data = "비스토")
        k_btn37 = BT(text = "카스타", callback_data = "카스타")
        k_btn38 = BT(text = "캐피탈", callback_data = "캐피탈")
        mu = MU(inline_keyboard = [[k_btn1, k_btn2], [k_btn3, k_btn4], [k_btn5, k_btn6], [k_btn7, k_btn8],
                                   [k_btn9, k_btn10], [k_btn11, k_btn12], [k_btn13, k_btn14], [k_btn15, k_btn16],
                                   [k_btn17, k_btn18], [k_btn19, k_btn20], [k_btn21, k_btn22], [k_btn23, k_btn24],
                                   [k_btn25, k_btn26], [k_btn27, k_btn28], [k_btn29, k_btn30], [k_btn31, k_btn32],
                                   [k_btn33, k_btn34], [k_btn35, k_btn36], [k_btn37, k_btn38]])
        bot.sendMessage(id, "기아차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '쌍용':
        s_btn1 = BT(text = "티볼리", callback_data = "티볼리")
        s_btn2 = BT(text = "무쏘", callback_data = "무쏘")
        s_btn3 = BT(text = "렉스턴", callback_data = "렉스턴")
        s_btn4 = BT(text = "코란도", callback_data = "코란도")
        s_btn5 = BT(text = "카이런", callback_data = "카이런")
        s_btn6 = BT(text = "코란도", callback_data = "코란도")
        s_btn7 = BT(text = "액티언", callback_data = "액티언")
        s_btn8 = BT(text = "체어맨", callback_data = "체어맨")
        s_btn9 = BT(text = "렉스턴", callback_data = "렉스턴")
        s_btn10 = BT(text = "로디우스", callback_data = "로디우스")
        mu = MU(inline_keyboard = [[s_btn1, s_btn2], [s_btn3, s_btn4], [s_btn5, s_btn6], [s_btn7, s_btn8],
                                   [s_btn9, s_btn10]])
        bot.sendMessage(id, "쌍용차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '제네시스':
        g_btn1 = BT(text = "BH330", callback_data = "BH330")
        g_btn2 = BT(text = "BH380", callback_data = "BH380")
        g_btn3 = BT(text = "GP500", callback_data = "GP500")
        g_btn4 = BT(text = "G330", callback_data = "G330")
        g_btn5 = BT(text = "G380", callback_data = "G380")
        g_btn6 = BT(text = "G90", callback_data = "G90")
        g_btn7 = BT(text = "G80", callback_data = "G80")
        g_btn8 = BT(text = "EQ900", callback_data = "EQ900")
        g_btn9 = BT(text = "BH460", callback_data = "BH460")
        g_btn10 = BT(text = "3.8", callback_data = "3.8")
        g_btn11 = BT(text = "GV70", callback_data = "GV70")
        g_btn12 = BT(text = "GV80", callback_data = "GV80")
        g_btn13 = BT(text = "G70", callback_data = "G70")
        g_btn14 = BT(text = "GP380", callback_data = "GP380")
        mu = MU(inline_keyboard = [[g_btn1, g_btn2], [g_btn3, g_btn4], [g_btn5, g_btn6], [g_btn7, g_btn8],
                                   [g_btn9, g_btn10], [g_btn11, g_btn12], [g_btn13, g_btn14]])
        bot.sendMessage(id, "제네시스차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '한국GM':
        m_btn1 = BT(text = "토스카", callback_data = "토스카")
        m_btn2 = BT(text = "쉐보레 말리부", callback_data = " 쉐보레 말리부")
        m_btn3 = BT(text = "쉐보레 카마로", callback_data = "쉐보레 카마로")
        m_btn4 = BT(text = "카마로", callback_data = "카마로")
        m_btn5 = BT(text = "쉐보레 콜벳", callback_data = "쉐보레 콜벳")
        m_btn6 = BT(text = "쉐보레 트래버스", callback_data = "쉐보레 트래버스")
        m_btn7 = BT(text = "쉐보레 임팔라", callback_data = "쉐보레 임팔라")
        m_btn8 = BT(text = "스테이츠맨", callback_data = "스테이츠맨")
        m_btn9 = BT(text = "베리타스", callback_data = "베리타스")
        m_btn10 = BT(text = "아카디아", callback_data = "아카디아")
        m_btn11 = BT(text = "알페온", callback_data = "알페온")
        m_btn12 = BT(text = "윈스톰", callback_data = "윈스톰")
        m_btn13 = BT(text = "쉐보레 캡티바", callback_data = "쉐보레 캡티바")
        m_btn14 = BT(text = "쉐보레 올란도", callback_data = "쉐보레 올란도")
        m_btn15 = BT(text = "라세티", callback_data = "라세티")
        m_btn16 = BT(text = "쉐보레 크루즈", callback_data = "쉐보레 크루즈")
        m_btn17 = BT(text = "레조", callback_data = "레조")
        m_btn18 = BT(text = "G2X", callback_data = "G2X")
        m_btn19 = BT(text = "쉐보레 말리부", callback_data = "쉐보레 말리부")
        m_btn20 = BT(text = "토스카", callback_data = "토스카")
        m_btn21 = BT(text = "프린스", callback_data = "프린스")
        m_btn22 = BT(text = "쉐보레 아베오", callback_data = "쉐보레 아베오")
        m_btn23 = BT(text = "쉐보레 트랙스", callback_data = "쉐보레 트랙스")
        m_btn24 = BT(text = "쉐보레 이쿼녹스", callback_data = "쉐보레 이쿼녹스")
        m_btn25 = BT(text = "젠트라", callback_data = "젠트라")
        m_btn26 = BT(text = "쉐보레 볼트", callback_data = "쉐보레 볼트")
        m_btn27 = BT(text = "쉐보레 트레일블레이저", callback_data = "쉐보레 트레일블레이저")
        m_btn28 = BT(text = "쉐보레 스파크", callback_data = "쉐보레 스파크")
        m_btn29 = BT(text = "스파크", callback_data = "스파크")
        m_btn30 = BT(text = "마티즈", callback_data = "마티즈")
        m_btn31 = BT(text = "티코", callback_data = "티코")
        m_btn32 = BT(text = "라보", callback_data = "라보")
        m_btn33 = BT(text = "다마스", callback_data = "다마스")
        mu = MU(inline_keyboard = [[m_btn1, m_btn2], [m_btn3, m_btn4], [m_btn5, m_btn6], [m_btn7, m_btn8],
                                   [m_btn9, m_btn10], [m_btn11, m_btn12], [m_btn13, m_btn14], [m_btn15, m_btn16],
                                   [m_btn17, m_btn18], [m_btn19, m_btn20], [m_btn21, m_btn22], [m_btn23, m_btn24],
                                   [m_btn25, m_btn26], [m_btn27, m_btn28], [m_btn29, m_btn30], [m_btn31, m_btn32, m_btn33]])
        bot.sendMessage(id, "한국GM차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '르노삼성':
        r_btn1 = BT(text = "QM6", callback_data = "QM6")
        r_btn2 = BT(text = "QM5", callback_data = "QM5")
        r_btn3 = BT(text = "QM3", callback_data = "QM3")
        r_btn4 = BT(text = "SM7", callback_data = "SM7")
        r_btn5 = BT(text = "SM5", callback_data = "SM5")
        r_btn6 = BT(text = "SM6", callback_data = "SM6")
        r_btn7 = BT(text = "SM3", callback_data = "SM3")
        r_btn8 = BT(text = "클리오", callback_data = "클리오")
        r_btn9 = BT(text = "XM3", callback_data = "XM3")
        r_btn10 = BT(text = "조에", callback_data = "조에")
        r_btn11 = BT(text = "캡처", callback_data = "캡처")
        r_btn12 = BT(text = "트위지", callback_data = "트위지")
        mu = MU(inline_keyboard = [[r_btn1, r_btn2], [r_btn3, r_btn4], [r_btn5, r_btn6], [r_btn7, r_btn8],
                                   [r_btn9, r_btn10], [r_btn11, r_btn12]])
        bot.sendMessage(id, "르노삼성차 모델을 선택해주세요", reply_markup = mu)

  
    elif btn_data in car_arr:
        payinfo = carinfo(btn_data)
        # bot.send_message(chat_id=id, text=f'{pricelist}')
        bot.sendMessage(chat_id=id, text='찾으신 가격정보입니다' + '\n' +
                        '최고가 & 최저가 정보\n------------------------------\n' + 
                        '제조사, 모델, 차종, 가격(만원), 연식, 주행거리(km), 연료, 변속기, 연비(km), 배기량(cc)' + '\n' + '\n' +
                        '평균가 정보\n------------------------------\n' +
                        '가격(만원), 연식, 주행거리(km), 연비(km), 배기량(cc)' + '\n' + '\n' +
                        payinfo)
        bot.sendMessage(chat_id=id, text='차차봇을 이용해주셔서 감사합니다!\n' + '다른 정보가 궁금하시면 *차차야*라고 불러주세요')


In [ ]:
def query_ans(msg):
    btn_data = msg["data"]

    if btn_data == '현대':
        h_btn1 = BT(text = "쿠페", callback_data = "쿠페")
        h_btn2 = BT(text = "제네시스", callback_data = "제네시스")
        h_btn3 = BT(text = "그랜저", callback_data = "그랜저")
        h_btn4 = BT(text = "에쿠스", callback_data = "에쿠스")
        h_btn6 = BT(text = "베라크루즈", callback_data = "베라크루즈")
        h_btn7 = BT(text = "팰리세이드", callback_data = "팰리세이드")
        h_btn9 = BT(text = "아슬란", callback_data = "아슬란")
        h_btn10 = BT(text = "맥스크루즈", callback_data = "맥스크루즈")
        h_btn11 = BT(text = "쏘나타", callback_data = "쏘나타")
        h_btn12 = BT(text = "아반떼", callback_data = "아반떼")
        h_btn13 = BT(text = "다이너스티", callback_data = "다이너스티")
        h_btn14 = BT(text = "갤로퍼", callback_data = "갤로퍼")
        h_btn15 = BT(text = "테라칸", callback_data = "테라칸")
        h_btn16 = BT(text = "트라제XG", callback_data = "트라제XG")
        h_btn17 = BT(text = "투스카니", callback_data = "투스카니")
        h_btn18 = BT(text = "싼타페", callback_data = "싼타페")
        h_btn20 = BT(text = "스타렉스", callback_data = "스타렉스")
        h_btn21 = BT(text = "스타리아", callback_data = "스타리아")
        h_btn22 = BT(text = "i30", callback_data = "i30")
        h_btn23 = BT(text = "벨로스터", callback_data = "벨로스터")
        h_btn24 = BT(text = "투싼", callback_data = "투싼")
        h_btn25 = BT(text = "코나", callback_data = "코나")
        h_btn26 = BT(text = "i40", callback_data = "i40")
        h_btn27 = BT(text = "엘란트라", callback_data = "엘란트라")
        h_btn28 = BT(text = "마르샤", callback_data = "마르샤")
        h_btn29 = BT(text = "아이오닉", callback_data = "아이오닉")
        h_btn30 = BT(text = "그라나다", callback_data = "그라나다")
        h_btn31 = BT(text = "티뷰론", callback_data = "티뷰론")
        h_btn32 = BT(text = "엑센트", callback_data = "엑센트")
        h_btn33 = BT(text = "베르나", callback_data = "베르나")
        h_btn34 = BT(text = "클릭", callback_data = "클릭")
        h_btn35 = BT(text = "베뉴", callback_data = "베뉴")
        h_btn36 = BT(text = "뉴엘란트라", callback_data = "뉴엘란트라")
        h_btn37 = BT(text = "포니", callback_data = "포니")
        h_btn38 = BT(text = "아이오닉5", callback_data = "아이오닉5")
        h_btn39 = BT(text = "넥쏘", callback_data = "넥쏘")
        h_btn40 = BT(text = "아토스", callback_data = "아토스")
        h_btn41 = BT(text = "엑셀", callback_data = "엑셀")
        mu = MU(inline_keyboard = [[h_btn1, h_btn2], [h_btn3, h_btn4], [h_btn6, h_btn7],
                                   [h_btn9, h_btn10], [h_btn11, h_btn12], [h_btn13, h_btn14], [h_btn15, h_btn16],
                                   [h_btn17, h_btn18], [h_btn41, h_btn20], [h_btn21, h_btn22], [h_btn23, h_btn24],
                                   [h_btn25, h_btn26], [h_btn27, h_btn28], [h_btn29, h_btn30], [h_btn31, h_btn32],
                                   [h_btn33, h_btn34], [h_btn35, h_btn36], [h_btn37, h_btn38], [h_btn39, h_btn40]])
        bot.sendMessage(id, "현대차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '기아':
        k_btn1 = BT(text = "모닝", callback_data = "모닝")
        k_btn2 = BT(text = "K7", callback_data = "K7")
        k_btn3 = BT(text = "K9", callback_data = "K9")
        k_btn4 = BT(text = "프라이드", callback_data = "프라이드")
        k_btn5 = BT(text = "모하비", callback_data = "모하비")
        k_btn6 = BT(text = "카니발", callback_data = "카니발")
        k_btn7 = BT(text = "오피러스", callback_data = "오피러스")
        k_btn8 = BT(text = "아만티", callback_data = "아만티")
        k_btn9 = BT(text = "K8", callback_data = "K8")
        k_btn10 = BT(text = "K5", callback_data = "K5")
        k_btn11 = BT(text = "로체", callback_data = "로체")
        k_btn12 = BT(text = "스팅어", callback_data = "스팅어")
        k_btn13 = BT(text = "쏘렌토", callback_data = "쏘렌토")
        k_btn14 = BT(text = "엔터프라이즈", callback_data = "엔터프라이즈")
        k_btn15 = BT(text = "쏘울", callback_data = "쏘울")
        k_btn16 = BT(text = "포르테", callback_data = "포르테")
        k_btn17 = BT(text = "K3", callback_data = "K3")
        k_btn18 = BT(text = "포텐샤", callback_data = "포텐샤")
        k_btn19 = BT(text = "카렌스", callback_data = "카렌스")
        k_btn20 = BT(text = "크레도스", callback_data = "크레도스")
        k_btn21 = BT(text = "스포티지", callback_data = "스포티지")
        k_btn22 = BT(text = "옵티마", callback_data = "옵티마")
        k_btn23 = BT(text = "레토나", callback_data = "레토나")
        k_btn24 = BT(text = "콩코드", callback_data = "콩코드")
        k_btn25 = BT(text = "엑스트렉", callback_data = "엑스트렉")
        k_btn26 = BT(text = "엘란", callback_data = "엘란")
        k_btn27 = BT(text = "쎄라토", callback_data = "쎄라토")
        k_btn28 = BT(text = "스토닉", callback_data = "스토닉")
        k_btn29 = BT(text = "셀토스", callback_data = "셀토스")
        k_btn30 = BT(text = "니로", callback_data = "니로")
        k_btn31 = BT(text = "세피아", callback_data = "세피아")
        k_btn32 = BT(text = "스펙트라", callback_data = "스펙트라")
        k_btn33 = BT(text = "레이", callback_data = "레이")
        k_btn34 = BT(text = "모닝밴", callback_data = "모닝밴")
        k_btn35 = BT(text = "레이밴", callback_data = "레이밴")
        k_btn36 = BT(text = "비스토", callback_data = "비스토")
        k_btn37 = BT(text = "카스타", callback_data = "카스타")
        k_btn38 = BT(text = "캐피탈", callback_data = "캐피탈")
        mu = MU(inline_keyboard = [[k_btn1, k_btn2], [k_btn3, k_btn4], [k_btn5, k_btn6], [k_btn7, k_btn8],
                                   [k_btn9, k_btn10], [k_btn11, k_btn12], [k_btn13, k_btn14], [k_btn15, k_btn16],
                                   [k_btn17, k_btn18], [k_btn19, k_btn20], [k_btn21, k_btn22], [k_btn23, k_btn24],
                                   [k_btn25, k_btn26], [k_btn27, k_btn28], [k_btn29, k_btn30], [k_btn31, k_btn32],
                                   [k_btn33, k_btn34], [k_btn35, k_btn36], [k_btn37, k_btn38]])
        bot.sendMessage(id, "기아차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '쌍용':
        s_btn1 = BT(text = "티볼리", callback_data = "티볼리")
        s_btn2 = BT(text = "무쏘", callback_data = "무쏘")
        s_btn3 = BT(text = "렉스턴", callback_data = "렉스턴")
        s_btn4 = BT(text = "코란도", callback_data = "코란도")
        s_btn5 = BT(text = "카이런", callback_data = "카이런")
        s_btn6 = BT(text = "액티언", callback_data = "액티언")
        s_btn7 = BT(text = "체어맨", callback_data = "체어맨")
        s_btn8 = BT(text = "로디우스", callback_data = "로디우스")
        mu = MU(inline_keyboard = [[s_btn1, s_btn2], [s_btn3, s_btn4], [s_btn5, s_btn6], [s_btn7, s_btn8]])
        bot.sendMessage(id, "쌍용차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '제네시스':
        g_btn1 = BT(text = "BH330", callback_data = "BH330")
        g_btn2 = BT(text = "BH380", callback_data = "BH380")
        g_btn3 = BT(text = "GP500", callback_data = "GP500")
        g_btn4 = BT(text = "G330", callback_data = "G330")
        g_btn5 = BT(text = "G380", callback_data = "G380")
        g_btn6 = BT(text = "G90", callback_data = "G90")
        g_btn7 = BT(text = "G80", callback_data = "G80")
        g_btn8 = BT(text = "EQ900", callback_data = "EQ900")
        g_btn9 = BT(text = "BH460", callback_data = "BH460")
        g_btn10 = BT(text = "GP380", callback_data = "GP380")
        g_btn11 = BT(text = "GV70", callback_data = "GV70")
        g_btn12 = BT(text = "GV80", callback_data = "GV80")
        g_btn13 = BT(text = "G70", callback_data = "G70")
        mu = MU(inline_keyboard = [[g_btn1, g_btn2], [g_btn3, g_btn4], [g_btn5, g_btn6], [g_btn7, g_btn8],
                                   [g_btn9, g_btn10], [g_btn11, g_btn12, g_btn13]])
        bot.sendMessage(id, "제네시스차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '한국GM':
        m_btn1 = BT(text = "토스카", callback_data = "토스카")
        m_btn2 = BT(text = "말리부", callback_data = "말리부")
        m_btn3 = BT(text = "카마로", callback_data = "카마로")
        m_btn4 = BT(text = "콜벳", callback_data = "콜벳")
        m_btn5 = BT(text = "트래버스", callback_data = "트래버스")
        m_btn6 = BT(text = "임팔라", callback_data = "임팔라")
        m_btn7 = BT(text = "스테이츠맨", callback_data = "스테이츠맨")
        m_btn8 = BT(text = "베리타스", callback_data = "베리타스")
        m_btn9 = BT(text = "아카디아", callback_data = "아카디아")
        m_btn10 = BT(text = "알페온", callback_data = "알페온")
        m_btn11 = BT(text = "윈스톰", callback_data = "윈스톰")
        m_btn12 = BT(text = "캡티바", callback_data = "캡티바")
        m_btn13 = BT(text = "올란도", callback_data = "올란도")
        m_btn14 = BT(text = "라세티", callback_data = "라세티")
        m_btn15 = BT(text = "크루즈", callback_data = "크루즈")
        m_btn16 = BT(text = "레조", callback_data = "레조")
        m_btn17 = BT(text = "G2X", callback_data = "G2X")
        m_btn18 = BT(text = "프린스", callback_data = "프린스")
        m_btn19 = BT(text = "아베오", callback_data = "아베오")
        m_btn20 = BT(text = "트랙스", callback_data = "트랙스")
        m_btn21 = BT(text = "이쿼녹스", callback_data = "이쿼녹스")
        m_btn22 = BT(text = "젠트라", callback_data = "젠트라")
        m_btn23 = BT(text = "볼트", callback_data = "볼트")
        m_btn24 = BT(text = "트레일블레이저", callback_data = "트레일블레이저")
        m_btn25 = BT(text = "스파크", callback_data = "스파크")
        m_btn26 = BT(text = "마티즈", callback_data = "마티즈")
        m_btn27 = BT(text = "티코", callback_data = "티코")
        m_btn28 = BT(text = "라보", callback_data = "라보")
        m_btn29 = BT(text = "다마스", callback_data = "다마스")
        mu = MU(inline_keyboard = [[m_btn1, m_btn2], [m_btn3, m_btn4], [m_btn5, m_btn6], [m_btn7, m_btn8],
                                   [m_btn9, m_btn10], [m_btn11, m_btn12], [m_btn13, m_btn14], [m_btn15, m_btn16],
                                   [m_btn17, m_btn18], [m_btn19, m_btn20], [m_btn21, m_btn22], [m_btn23, m_btn24],
                                   [m_btn25, m_btn26], [m_btn27, m_btn28, m_btn29]])
        bot.sendMessage(id, "한국GM차 모델을 선택해주세요", reply_markup = mu)

    elif btn_data == '르노삼성':
        r_btn1 = BT(text = "QM6", callback_data = "QM6")
        r_btn2 = BT(text = "QM5", callback_data = "QM5")
        r_btn3 = BT(text = "QM3", callback_data = "QM3")
        r_btn4 = BT(text = "SM7", callback_data = "SM7")
        r_btn5 = BT(text = "SM5", callback_data = "SM5")
        r_btn6 = BT(text = "SM6", callback_data = "SM6")
        r_btn7 = BT(text = "SM3", callback_data = "SM3")
        r_btn8 = BT(text = "클리오", callback_data = "클리오")
        r_btn9 = BT(text = "XM3", callback_data = "XM3")
        r_btn10 = BT(text = "조에", callback_data = "조에")
        r_btn11 = BT(text = "캡처", callback_data = "캡처")
        r_btn12 = BT(text = "트위지", callback_data = "트위지")
        mu = MU(inline_keyboard = [[r_btn1, r_btn2], [r_btn3, r_btn4], [r_btn5, r_btn6], [r_btn7, r_btn8],
                                   [r_btn9, r_btn10], [r_btn11, r_btn12]])
        bot.sendMessage(id, "르노삼성차 모델을 선택해주세요", reply_markup = mu)

  
    elif btn_data in car_arr:
        payinfo = carinfo(btn_data)
        # bot.send_message(chat_id=id, text=f'{pricelist}')
        bot.sendMessage(chat_id=id, text='찾으신 가격정보입니다' + '\n' +
                        '최고가 & 최저가 정보\n------------------------------\n' + 
                        '제조사, 모델, 차종, 가격(만원), 연식, 주행거리(km), 연료, 변속기, 연비(km), 배기량(cc)' + '\n' + '\n' +
                        '평균가 정보\n------------------------------\n' +
                        '가격(만원), 연식, 주행거리(km), 연비(km), 배기량(cc)' + '\n' + '\n' +
                        payinfo)
        bot.sendMessage(chat_id=id, text='차차봇을 이용해주셔서 감사합니다!\n' + '다른 정보가 궁금하시면 *차차야*라고 불러주세요')

In [ ]:
def predict(text):
    user_values = text.split()
    user_values = user_values[1:10]
    bot.send_message(chat_id = id, text = f'입력해주신 \n\n{user_values}\n\n 정보를 토대로 가격 예측중 입니다 잠시만 기다려주세요')
    
    bot.send_animation(animation=open('./img.gif', 'rb'), chat_id = id, duration = 5).deleteLater()
    
    
    # 제조사 모델 차종 연식 주행거리 연료 변속기 연비 배기량
    user_df = df.drop(['가격(만원)'], axis = 1)
    user_df.append(pd.Series(user_values, index=user_df.columns), ignore_index=True)
    hot_incoder = pd.get_dummies(df)
    hot_incoder1 = pd.get_dummies(user_df)
    
    features = hot_incoder1.loc[:, '연식':'변속기_오토']
    X = features.values
    y = hot_incoder['가격(만원)']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    logreg = RandomForestRegressor(oob_score=True, n_jobs=-1, random_state=42)
    logreg.fit(X_train, y_train)
    train_pred = logreg.predict(X_test)
    
    bot.send_message(id, f'{round(accuracy_score(y_test, train_pred), 2) * 100}% 정확도의 머신러닝을 통한 중고차 예측 가격은 {round(train_pred[len(train_pred)-1])}만원 입니다.')

In [ ]:
def predict(text):

    user_values = text.split()
    user_values = user_values[0:10]

    # 제조사 모델 차종 연식 주행거리 연료 변속기 연비 배기량
    user_df = df.drop(['가격(만원)'], axis = 1)
    user_df.append(pd.Series(user_values, index=user_df.columns), ignore_index=True)
    hot_incoder = pd.get_dummies(df)
    hot_incoder1 = pd.get_dummies(user_df)
    
    features = hot_incoder1.loc[:, '연식':'변속기_오토']
    X = features.values
    y = hot_incoder['가격(만원)']
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
    logreg = RandomForestRegressor(oob_score=True, n_jobs=-1, random_state=42)
    logreg.fit(X_train, y_train)
    train_pred = logreg.predict(X_test)
    mess = f'{round(mean_squared_error(y_test, train_pred), 2) * 100}% 정확도로 자동차 예상 가격은 {round(train_pred[len(train_pred)-1])}만원 입니다.'

    return mess

In [ ]:
# 제조사 모델 차종 연식 주행거리 연료 변속기 연비 배기량
predict('기아 K5 중형 2020 10382 가솔린 오토 13 0')

'100.0% 정확도의 머신러닝을 통한 중고차 예측 가격은 3947만원 입니다.'

In [ ]:
predict('제네시스 G80 대형 2018 60000 디젤 오토 12 1000')

'100.0% 정확도의 머신러닝을 통한 중고차 예측 가격은 2449만원 입니다.'

In [ ]:
# def predict(text):

#     user_values = text.split()
#     user_values = user_values[0:10]

#     # 제조사 모델 차종 연식 주행거리 연료 변속기 연비 배기량
#     user_df = df.drop(['가격(만원)'], axis = 1)
#     hot_incoder = pd.get_dummies(df)
#     hot_incoder1 = pd.get_dummies(user_df)
    
#     features = hot_incoder1.loc[:, '연식':'변속기_오토']
#     X = features.values
#     y = hot_incoder['가격(만원)']
#     X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0)
#     logreg = RandomForestRegressor(oob_score=True, n_jobs=-1, random_state=42)
#     logreg.fit(X_train, y_train)
#     train_pred = logreg.predict(X_test)
#     mess = f'{round(mean_squared_error(y_test, train_pred), 2) * 100}% 정확도로 자동차 예상 가격은 {round(train_pred[len(train_pred)-1])}만원 입니다.'

#     return mess

In [ ]:
def predict(text):
    user_values = text.split()
    user_values = user_values[0:10]

        # 제조사 모델 차종 연식 주행거리 연료 변속기 연비 배기량
    input_info = pd.DataFrame(user_values).transpose()
    user_df = df.drop(['가격(만원)'], axis = 1)
    input_info.columns = user_df.columns
    user_df = pd.concat([user_df, input_info], ignore_index=True)
    # 컬럼별 type을 df함수와 동일하게
    user_df = user_df.astype({'연식': 'int'})
    user_df = user_df.astype({'연비(km)' : 'float'})
    user_df = user_df.astype({'주행거리(km)': 'int'})
    user_df = user_df.astype({'배기량(cc)' : 'float'})
    hot_incoder = pd.get_dummies(df)
    hot_incoder1 = pd.get_dummies(user_df)
    # 입력값 추출
    my_car = hot_incoder1.tail(1)
    
    features = hot_incoder.drop(['가격(만원)'], axis=1)
    X = features.values
    y = hot_incoder['가격(만원)']

    logreg = RandomForestRegressor(oob_score=True, n_jobs=-1, random_state=42)
    logreg.fit(X, y)
    train_pred = logreg.predict(my_car)
    mess = f'{round(logreg.score(X, y), 2) * 100}% 정확도의 머신러닝을 통한 중고차 예측 가격은 {round(train_pred[0])}만원 입니다.'

    return mess

In [ ]:
predict('제네시스 G80 대형 2020 10000 가솔린 오토 18 3000')

'100.0% 정확도의 머신러닝을 통한 중고차 예측 가격은 4443만원 입니다.'

In [ ]:
# def predict(text):
#     user_values = text.split()
#     user_values = user_values[1:10]
#     bot.send_message(chat_id = id, text = f'입력해주신 \n\n{user_values}\n\n 정보를 토대로 가격 예측중 입니다 잠시만 기다려주세요')
#         # 제조사 모델 차종 연식 주행거리 연료 변속기 연비 배기량
#     input_info = pd.DataFrame(user_values).transpose()
#     user_df = df.drop(['가격(만원)'], axis = 1)
#     input_info.columns = user_df.columns
#     user_df = pd.concat([user_df, input_info], ignore_index=True)
#     # 컬럼별 type을 df함수와 동일하게
#     user_df = user_df.astype({'연식': 'int'})
#     user_df = user_df.astype({'연비(km)' : 'float'})
#     user_df = user_df.astype({'주행거리(km)': 'int'})
#     user_df = user_df.astype({'배기량(cc)' : 'float'})
#     hot_incoder = pd.get_dummies(df)
#     hot_incoder1 = pd.get_dummies(user_df)
#     # 입력값 추출
#     my_car = hot_incoder1.tail(1)
    
#     features = hot_incoder.drop(['가격(만원)'], axis=1)
#     X = features.values
#     y = hot_incoder['가격(만원)']

#     logreg = RandomForestRegressor(oob_score=True, n_jobs=-1, random_state=42)
#     logreg.fit(X, y)
#     train_pred = logreg.predict(my_car)
#     bot.send_message(id, f'{round(logreg.score(X, y), 2) * 100}% 정확도의 머신러닝을 통한 중고차 예측 가격은 {round(train_pred[0])}만원 입니다.')